In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LayerNormalization
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import numpy as np
import glob


2023-09-30 20:25:47.159138: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-09-30 20:25:47.159349: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-09-30 20:25:47.159574: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-30 20:25:47.198674: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class HomelessModel(Model):
    def __init__(self):
        super(HomelessModel, self).__init__()
        self.layerz = []
        self.layerz.append(Conv2D(16, (2, 2), activation='relu', input_shape=(192, 320, 3)))
        self.layerz.append(MaxPooling2D(2, 2))
        self.layerz.append(Conv2D(32, (3, 3), activation='relu'))
        self.layerz.append(Flatten())
        self.layerz.append(Dense(128, activation='relu'))
        self.layerz.append(Dense(6, activation='softmax'))

    def call(self, inputs):
        x = inputs
        for layer in self.layerz:
            x = layer(x)
        return x

# Create an instance of your custom model
model = HomelessModel()

2023-09-30 20:10:08.172543: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-30 20:10:08.176927: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-30 20:10:08.176963: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-30 20:10:08.179453: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-30 20:10:08.179498: I tensorflow/compile

In [3]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [4]:
train_images = []
train_labels = []

def loadImages(images, label):
    for img_path in images:
        train_images.append(img_to_array(load_img(img_path, target_size=(192, 320))))
        train_labels.append(label)
        
# Define the label once
labelA = np.array([1, 0.11, 0.35, 0.34, 0.16, 0.42])  # A, C, E, I, R, S
labelC = np.array([0.11, 1, 0.68, 0.16, 0.36, 0.38])  # A, C, E, I, R, S
labelE = np.array([0.35, 0.68, 1, 0.21, 0.3, 0.54])  # A, C, E, I, R, S
labelI = np.array([0.34, 0.16, 0.21, 1, 0.46, 0.3])  # A, C, E, I, R, S
labelR = np.array([0.16, 0.36, 0.3, 0.46, 1, 0.21])  # A, C, E, I, R, S
labelS = np.array([0.42, 0.38, 0.54, 0.3, 0.21, 1])  # A, C, E, I, R, S
labelA /= sum(labelA)
labelC /= sum(labelC)
labelE /= sum(labelE)
labelI /= sum(labelI)
labelR /= sum(labelR)
labelS /= sum(labelS)


image_filesA = glob.glob('Artistic/*.png')
image_filesC = glob.glob('Conventional/*.png')
image_filesE = glob.glob('Enterprising/*.png')
image_filesI = glob.glob('Investigative/*.png')
image_filesR = glob.glob('Realistic/*.png')
image_filesS = glob.glob('Social/*.png')

loadImages(image_filesA, labelA)
loadImages(image_filesC, labelC)
loadImages(image_filesE, labelE)
loadImages(image_filesI, labelI)
loadImages(image_filesR, labelR)
loadImages(image_filesS, labelS)

reduce_lr_callback = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.7,
    patience=20,
    min_lr=1e-6
)

train_images = np.array(train_images).reshape(len(train_labels), 192, 320, 3)
train_labels = np.array(train_labels)

def learn():
    x_train, x_test, y_train, y_test = train_test_split(train_images, train_labels, test_size=0.2, shuffle=True)
    
    history = model.fit(
        x_train,
        y_train,
        epochs=50,
        batch_size=4,
        validation_data=(x_test, y_test),
        callbacks=[reduce_lr_callback],
        verbose=False
    )
    
    test_loss, test_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {test_accuracy*100:.2f}%")

In [5]:
for i in range(10):
    learn()
    model.predict(np.array(img_to_array(load_img("Conventional/5.png", target_size=(192, 320)))).reshape(1, 192, 320, 3))

2023-09-30 20:10:10.788288: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2023-09-30 20:10:11.509627: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb9a6720580 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-30 20:10:11.509671: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1050, Compute Capability 6.1
2023-09-30 20:10:11.514174: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-30 20:10:11.604095: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 [==============================] - 0s 344ms/step - loss: 1.8283 - accuracy: 0.5833
Test accuracy: 58.33%
1/1 [==============================] - 0s 38ms/step - loss: 1.6803 - accuracy: 0.9167
Test accuracy: 91.67%
1/1 [==============================] - 0s 37ms/step - loss: 1.6180 - accuracy: 0.9167
Test accuracy: 91.67%
1/1 [==============================] - 0s 41ms/step - loss: 1.5950 - accuracy: 0.9167
Test accuracy: 91.67%
1/1 [==============================] - 0s 39ms/step - loss: 1.6133 - accuracy: 1.0000
Test accuracy: 100.00%
1/1 [==============================] - 0s 39ms/step - loss: 1.6320 - accuracy: 1.0000
Test accuracy: 100.00%
1/1 [==============================] - 0s 38ms/step - loss: 1.6168 - accuracy: 0.9167
Test accuracy: 91.67%
1/1 [==============================] - 0s 39ms/step - loss: 1.6211 - accuracy: 1.0000
Test accuracy: 100.00%
1/1 [==============================] - 0s 37ms/step - loss: 1.6156 - accuracy: 1.0000
Test accuracy: 100.00%
1/1 [==================

In [6]:
model.save("HomelessModel.h5")

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


NotImplementedError: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.

In [7]:
model = keras.models.load_model('HomelessModel')


OpError: HomelessModel/variables/variables.data-00000-of-00001; No such file or directory

In [5]:
model.predict(np.array(img_to_array(load_img("Investigative/4.png", target_size=(192, 320)))).reshape(1, 192, 320, 3))

1/1 [==============================] - 0s 26ms/step


array([[0.13797662, 0.06454045, 0.08711764, 0.40100846, 0.18834011,
        0.12101664]], dtype=float32)